# Importações

In [ ]:
import pandas as pd 
import numpy as np

# Importar dados do PostgreSQL

In [ ]:
# Como a Api ainda não esta a funcionar, vou importar os dados de um csv
df = pd.read_csv('../all_bike_counts.csv', parse_dates=['detected'], dtype='int32')
df = df.set_index('detected')
df.info()

In [ ]:
df_loc = pd.read_csv('../all_counter_locations.csv')
df_loc.info()
print()

# Analises

In [ ]:
df.index

In [ ]:
df.tail()

In [ ]:
df.loc['2022'].tail(2)

In [ ]:
df.resample('D').first().dropna().index[[0, -1]]

In [ ]:
df.resample('D').first().dropna().iloc[[0, -1]]

In [ ]:
# Seleciona o primeiro e o último dia com os contadores
df_mostrar = df.resample('D').first().dropna().iloc[[0, -1]]

# Identifica as colunas que representam os contadores e direções
contador_cols = [col for col in df.columns if col.startswith('direction_') or col.startswith('count_')]

# Identifica o primeiro e o último contador
primeiro_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[0]
ultimo_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[-1]

# Seleciona as colunas para o primeiro e o último contador
primeiro_contador_cols = [f'direction_{primeiro_contador}', f'count_{primeiro_contador}']
ultimo_contador_cols = [f'direction_{ultimo_contador}', f'count_{ultimo_contador}']

# Exibe as colunas desejadas
df_selecionado = df_mostrar[primeiro_contador_cols + ultimo_contador_cols]

# Exibe o resultado
df_selecionado


In [ ]:
df_loc.head(2)

### Identificar se há dias com valores a zéro

In [ ]:
location_ids = [2,3,4,8,9,12,13]  
contadores = [f"count_{loc}" for loc in location_ids]

In [ ]:
df_daily = df.resample('D').sum()

In [ ]:
# Identificar zeros na série
for contador in contadores:
    if contador in df_daily.columns:  
        series = df_daily[contador].copy()  
        zero_indices = series[series == 0].index  

        if not zero_indices.empty:
            print(f"Zeros encontrados no contador '{contador}' nas datas: {list(zero_indices)}")

São sempre os mesmos dias que estão a zéros em todos os contadores. 